# Leser inn bibliotek og genererer testdata

## Leser inn bibliotek og setter opp spark session

In [ ]:
import dapla as dp
import pyspark.sql.functions as F
from pyspark.sql.types import *
from datetime import datetime
spark

In [ ]:
%run ../../ssb_sparktools/processing/processing.py

In [ ]:
myspark = SparkSession.builder.getOrCreate()

## Lager testdata 

#### Hierarkiske data

In [ ]:
hierarki_schema = StructType([
                              StructField('persid', StringType(), False),
                              StructField('dato', StringType(), False),
                              StructField('arbeidsgiver',ArrayType(
                                                                      StructType([StructField('navn',StringType(),True),
                                                                                  StructField('adresse',StringType(),True),
                                                                                  StructField('ansatte', ArrayType(
                                                                                      StructType([StructField('navn', StringType(), True),
                                                                                                  StructField('adresse',StringType(),True)])))
                                                                                 ])
                                                                     )),
                              StructField('utdanning',ArrayType(
                                                                      StructType([StructField('utdanningsinstitusjon',StringType(),True),
                                                                                  StructField('adresse',StringType(),True),
                                                                                  StructField('utdanning', ArrayType(
                                                                                      StructType([StructField('fag', StringType(), True),
                                                                                                  StructField('eksamensdato',StringType(),True)])))
                                                                                 ])
                                                                     ))
                             ])
hierarkidata_raw = [('#ID1', '01Jan2020', [('Industri AS', 'Jernveien 24', [('Per', 'Storgata 3'),('Kari', 'Toppen 2')])], [('Mek Skole', 'Mek veien 1',[('Mekaniskefag', '21Jun2013'), ('Byggingeniør', '11Jun2018')])]),
                    ('#ID2', '02Mar2020', [('Lommerusk AS', 'Sliteveien 23', [('Espen', 'Ukjent'),('Ronny', 'Kaiegata 2')])], [('Harde Skole', 'Kjeppveien 10', [('Grunnskole', '19Jun2014')])]),
                    ('#ID3', '15Feb2020', [('Papir AS', 'Papirveien 24', [('Ole', 'Storgata 3'),('Siri', 'Toppen 3')])], [('Skogen Skole', 'Treveien 5', [('Papirfag', '21Jun2014'), ('Papiringeniør', '11Jun2012')])])]
hierarki_testdata = myspark.createDataFrame(hierarkidata_raw, hierarki_schema)

#### Skriver ut testdata som er laget

In [ ]:
hierarki_testdata.printSchema()

In [ ]:
hierarki_testdata.show()

# Test av funksjonen *cross_sectional*

### Uten spark_session som parameter

In [ ]:
dato_hendelsedata = hierarki_testdata.withColumn('dato', F.to_timestamp('dato', "ddMMMyyyy"))
tverrsnitt = cross_sectional(dato_hendelsedata, 'dato', ['persid'])

In [ ]:
tverrsnitt.show()

### Med spark_session som parameter

In [ ]:
dato_hendelsedata = hierarki_testdata.withColumn('dato', F.to_timestamp('dato', "ddMMMyyyy"))
tverrsnitt = cross_sectional(dato_hendelsedata, 'dato', ['persid'])

In [ ]:
tverrsnitt.show()

### Tar tverrsnitt på en gitt dato

In [ ]:
REFDATO = '2020-03-01 00:00:00'
referansedato = datetime.strptime(REFDATO, '%Y-%m-%d %H:%M:%S')

In [ ]:
tverrsnitt_co = cross_sectional(dato_hendelsedata, 'dato', ['persid'], coDate=referansedato)

In [ ]:
tverrsnitt_co.show()

# Test av funksjonen *unpack_parquet*

### Uten spark_session som parameter

In [ ]:
pakketut = unpack_parquet(hierarki_testdata)
for k in pakketut.keys():
    print(k)

### Med spark_session som parameter

In [ ]:
pakketut = unpack_parquet(hierarki_testdata, spark_session=myspark)
for k in pakketut.keys():
    print(k)

### Viser utpakket data

In [ ]:
pakketut['arbeidsgiver'].show()

In [ ]:
pakketut['arbeidsgiver_ansatte'].show()

In [ ]:
pakketut['utdanning'].show()

In [ ]:
pakketut['utdanning_utdanning-child'].show()

In [ ]:
print(f'Totalt antall dataframes: {len(pakketut)}')

In [ ]:
test_dict_less = unpack_parquet(hierarki_testdata, levels=1, spark_session=myspark)
print(f'Totalt antall dataframes: {len(test_dict_less)}')